# Transactions Fraud Detection

**Authors:** [Peter Macinec](https://github.com/pmacinec), [Timotej Zatko](https://github.com/timzatko)

## Preprocessing

In this jupyter notebook, we will preprocess the data. Preprocessed data can be then used for classification.

### Setup and reading the data

At first, we need to import libraries and set initial configs.

In [1]:
# Automatically reload imported modules
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

# Supress libraries deprecation import warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline

from src.preprocessing.transformers import *
from src.preprocessing.pandas_feature_union import PandasFeatureUnion
from src.preprocessing.pandas_one_hot_encoder import PandasOneHotEncoder
from src.preprocessing.pandas_simple_imputer import PandasSimpleImputer
from src.preprocessing.pandas_missing_indicator import PandasMissingIndicator

from src.dataset import load_data, split_and_save_processed_data

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

The data will be loaded using our function that optimizes data types of attributes (this loading saves a lot of memory):

In [5]:
df = load_data()

In [6]:
df.shape

(590540, 434)

### Define preprocessing pipeline

Our preprocessing will be done via preprocessing **pipeline**. Preprocessing with pipelines is commonly used to ensure reproducibility.

The steps of preprocessing are defined according to results of data analysis phase. Preprocessing pipeline can be devided into two branches - preprocessing of numeric and categorical attributes:

* **numerical attributes**
    * drop attributes with too many missing values (more than 50%)
    * fill in missing values (simply by using mean value)
    * normalize values
* **categorical attributes**
    * drop attributes with too many missing values (more than 50%)
    * transformation of emails
    * fill in missing values (simply by using most frequent value)
    * merge small categories into one `other` category
    * one-hot encoding (some algorithms work only with numerical values)

Also, missing values indicators have been added to both, numerical and categorical attributes.

In [7]:
categoric_features = df.select_dtypes(include=np.object).columns.to_list()
numeric_features = df.select_dtypes(exclude=np.object).columns.to_list()
label_feature = 'isFraud'

numeric_features.remove('TransactionID') # ID should not be used
numeric_features.remove(label_feature)

pipeline = PandasFeatureUnion([
    ('numeric_features', make_pipeline(
        SelectFeatures(numeric_features),
        FilterColumnsByCountOfMissingValues(0.5),
        PandasMissingIndicator(),
        PandasSimpleImputer(strategy='mean'),
        Normalizer()
    )),
    ('categoric_features', make_pipeline(
        SelectFeatures(categoric_features),
        FilterColumnsByCountOfMissingValues(0.5),
        EmailProviderTransform(['P_emaildomain', 'R_emaildomain']),
        PandasMissingIndicator(),
        PandasSimpleImputer(strategy='most_frequent'),
        MergeSmallCategories(),
        PandasOneHotEncoder()
    )),
    ('label_feature', make_pipeline(
        SelectFeatures([label_feature])
    ))
])

In [8]:
%%time
df_preprocessed = pipeline.fit_transform(df)

CPU times: user 12min 22s, sys: 39.6 s, total: 13min 1s
Wall time: 12min 58s


Let's check the new shape of dataframe after preprocessing:

In [9]:
df_preprocessed.shape

(590540, 440)

After preprocessing, number of samples was not changed. There is almost the same number of features, but we expect a lot of original features to be dropped because of missing values and a lot of new features being created using one-hot encoding. Let's check the values:

In [10]:
df_preprocessed

,TransactionDT,TransactionAmt,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D10,D11,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,D1_missing,D2_missing,D3_missing,D4_missing,D10_missing,D11_missing,D15_missing,V1_missing,V2_missing,V3_missing,V4_missing,V5_missing,V6_missing,V7_missing,V8_missing,V9_missing,V10_missing,V11_missing,V12_missing,V13_missing,V14_missing,V15_missing,V16_missing,V17_missing,V18_missing,V19_missing,V20_missing,V21_missing,V22_missing,V23_missing,V24_missing,V25_missing,V26_missing,V27_missing,V28_missing,V29_missing,V30_missing,V31_missing,V32_missing,V33_missing,V34_missing,V35_missing,V36_missing,V37_missing,V38_missing,V39_missing,V40_missing,V41_missing,V42_missing,V43_missing,V44_missing,V45_missing,V46_missing,V47_missing,V48_missing,V49_missing,V50_missing,V51_missing,V52_missing,V53_missing,V54_missing,V55_missing,V56_missing,V57_missing,V58_missing,V59_missing,V60_missing,V61_missing,V62_missing,V63_missing,V64_missing,V65_missing,V66_missing,V67_missing,V68_missing,V69_missing,V70_missing,V71_missing,V72_missing,V73_missing,V74_missing,V75_missing,V76_missing,V77_missing,V78_missing,V79_missing,V80_missing,V81_missing,V82_missing,V83_missing,V84_missing,V85_missing,V86_missing,V87_missing,V88_missing,V89_missing,V90_missing,V91_missing,V92_missing,V93_missing,V94_missing,V95_missing,V96_missing,V97_missing,V98_missing,V99_missing,V100_missing,V101_missing,V102_missing,V103_missing,V104_missing,V105_missing,V106_missing,V107_missing,V108_missing,V109_missing,V110_missing,V111_missing,V112_missing,V113_missing,V114_missing,V115_missing,V116_missing,V117_missing,V118_missing,V119_missing,V120_missing,V121_missing,V122_missing,V123_missing,V124_missing,V125_missing,V126_missing,V127_missing,V128_missing,V129_missing,V130_missing,V131_missing,V132_missing,V133_missing,V134_missing,V135_missing,V136_missing,V137_missing,V279_missing,V280_missing,V281_missing,V282_missing,V283_missing,V284_missing,V285_missing,V286_missing,V287_missing,V288_missing,V289_missing,V290_missing,V291_missing,V292_missing,V293_missing,V294_missing,V295_missing,V296_missing,V297_missing,V298_missing,V299_missing,V300_missing,V301_missing,V302_missing,V303_missing,V304_missing,V305_missing,V306_missing,V307_missing,V308_missing,V309_missing,V310_missing,V311_missing,V312_missing,V313_missing,V314_missing,V315_missing,V316_missing,V317_missing,V318_missing,V319_missing,V320_missing,V321_missing,card2_missing,card3_missing,card4_missing,card5_missing,card6_missing,addr1_missing,addr2_missing,M1_missing,M2_missing,M3_missing,M4_missing,M6_missing,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_W,ProductCD_other,card1_other,card2_490.0,card2_555.0,card2_other,card3_other,card4_mastercard,card4_other,card4_visa,card5_other,card6_credit,card6_debit,card6_other,addr1_other,addr2_87.0,addr2_other,P_emaildomain_anonymous,P_emaildomain_gmail,P_emaildomain_hotmail,P_emaildomain_nan,P_emaildomain_other,P_emaildomain_yahoo,M1_T,M1_other,M2_T,M2_other,M3_T,M3_other,M4_M0,M4_M1,M4_M2,M6_T,M6_other,isFraud
0,-1.577985,-0.278174,-0.098021,-0.092260,-0.037493,-0.059438,-0.21606,-0.112869,-0.046146,-0.053939,-0.208711,-0.054826,-0.087363,-0.047034,-0.243806,-0.147246,-0.510173,4.426463e-16,-0.3

All features should be encoded as numbers:

In [11]:
df_preprocessed.dtypes.value_counts()

float64    203
int64      199
uint8       37
bool         1
dtype: int64

### Train-test split

Before using the data for training machine learning algorithms, samples (rows) have to be splitted into two subsets - train set and test set. The size of test set will be simply 20% (as usually used):

In [12]:
%%time

split_and_save_processed_data(df_preprocessed, test_size=0.2)

Splitting the data...
Saving data...
CPU times: user 5min 37s, sys: 5.19 s, total: 5min 43s
Wall time: 5min 43s


All the subsets are stored for future usage in `data` folder in root directory.

### Conclusion

In this jupyter notebook, preprocessing phase was performed. Preprocessing pipeline was defined according to results of analysis. The way of preprocessing via pipeline was chosen to ensure reproducibility. After preprocessing, the data have been splitted into train-test subsets that will be used for training machine learning algorithms.